# Tomtom traffic flow rate

In [74]:
import networkx as nx
import pandas as pd
import requests
import json
from datetime import datetime
import requests
from math import sin, cos, sqrt, atan2, radians
import matplotlib.pyplot as plt
import time
import geopandas as gpd
%matplotlib inline

In [103]:
def get_flow_rate(lat, long):
    """
    Returns free flow traffic speed at a given coordinate in KMPH
    """
    CLIENT_KEY = ''
    url = "https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json?key={}&point={},{}&unit=KMPH&jsonp=jsonp".format(CLIENT_KEY,
                                                                                                               lat, long)
    r = requests.get(url)
    if r.status_code != 200:
        print(r.content)
        raise ValueError("Invalid URL Response for point {},{}".format(lat, long))
    time.sleep(0.2)
    return json.loads(r.content)['flowSegmentData']['freeFlowSpeed']

In [48]:
get_flow_rate(45.493454, -73.560172)

21

In [49]:
with open(r"./geojson_dump.json", 'r') as f:    
    data = f.read()

In [50]:
data_json = json.loads(data)

In [63]:
def distance_in_km(e1, e2):

    # approximate radius of earth in km
    R = 6373.0
    lat1 = radians(e1[1])
    lon1 = radians(e1[0])
    lat2 = radians(e2[1])
    lon2 = radians(e2[0])
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return distance

In [95]:
def middle_point(l):
    return l[int((len(l)-1)/2)][1], l[int((len(l)-1)/2)][0]

In [97]:
G = nx.DiGraph()
for feature in data_json['features']:
    sens_cir = feature['properties']['SENS_CIR']
    rate = get_flow_rate(*middle_point(feature['geometry']['coordinates']))
    for idx, coord in enumerate(feature['geometry']['coordinates']):
        coord = tuple(coord)
        if coord not in G:
            G.add_node(coord, **feature['properties'])
        if idx != 0:
            dist = distance_in_km(coord, last_node)
            tm = dist / rate
            if sens_cir == 0:
                G.add_edge(last_node, coord, length=dist, time=tm)
                G.add_edge(coord, last_node, length=dist, time=tm)
            elif sens_cir == 1:
                G.add_edge(last_node, coord, length=dist, time=tm)
            elif sens_cir == -1:
                G.add_edge(coord, last_node, length=dist, time=tm)
        last_node = coord

43
31
43
43
43
43
46
43
39
37
37
95
43
43
34


KeyboardInterrupt: 

In [79]:
test = gpd.read_file(r"./geojson_dump.json")

In [84]:
G = nx.Graph()
for feature in data_json['features']:
    # if feature['properties']['ARR_DRT'] == "Le Sud-Ouest":
    for idx, coord in enumerate(feature['geometry']['coordinates']):
        coord = tuple(coord)
        if coord not in G:
            G.add_node(coord, **feature['properties'])
        if idx != 0:
            G.add_edge(last_node, coord, length=distance_in_km(coord, last_node))
        last_node = coord

In [94]:
G = nx.DiGraph()
for feature in data_json['features']:
    # if feature['properties']['ARR_DRT'] == "Le Sud-Ouest":
    sens_cir = feature['properties']['SENS_CIR']
    print('\n')
    print(feature['geometry']['coordinates'])
    print(middle_point(feature['geometry']['coordinates']))
    if len(feature['geometry']['coordinates'])>2:
        break



[[-73.74081689158372, 45.51897292308536], [-73.7419541040326, 45.519703332189074]]
(45.51897292308536, -73.74081689158372)


[[-73.70623530423325, 45.527646493721306], [-73.70629430924254, 45.52759778866121], [-73.7066906006261, 45.52727777851528], [-73.70671238591936, 45.52725785084725], [-73.7067274208916, 45.527240095005574], [-73.70673392212828, 45.52722766433297], [-73.70674081687976, 45.5272073134621], [-73.70674125317629, 45.52718220468139], [-73.70673739506634, 45.52716673285305], [-73.70673249273064, 45.52715450278576], [-73.70671879388978, 45.52713346931191], [-73.7067027913941, 45.527117659522766], [-73.70598918338321, 45.52665575670475], [-73.70594008055075, 45.526626507550006], [-73.70592368938513, 45.52661987761266], [-73.70588955127046, 45.52661129978933], [-73.70586238092204, 45.52660895270981], [-73.70582259798886, 45.526609440797735], [-73.7058045516517, 45.52661226303902], [-73.7057696380703, 45.526622764971755], [-73.70575540622154, 45.526629180141576], [-73.70572

In [86]:
len(test)

47391